In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from utils.activations import softmax, relu

sns.set()

m = 24754  # no of examples
n = 784 # no of features 
num_class = 4 # no of classes
hidden_layer_units = 10 


## DataSource
X_train, Y_train is parsed from csv files 

In [2]:
X_train = pd.read_csv(os.path.join('data','train_data.csv'), header=None).add_prefix('Feature_')
Y_train = pd.read_csv(os.path.join('data', 'train_labels.csv'), header=None, names=["Label_0", "Label_1", "Label_2", "Label_3"])
X_train_transpose = X_train.T


## Layer and Sequence
We then implement Layer and Sequence as follow

In [3]:
def Layer(A_in, W, B, g):
    """
    :param A_in: shape(m,n) - input data
    :param W: shape(feature,units) - weight matrix, n0 feature  x units, 
    :param b: shape(units,1) - bias vector, n0 units x 1
    :param g: activation function(e.g sigmoid, relu, softmax, ...)
    :return:
    A_out: shape(m, units): output data - m x units
    """
    Z = np.matmul(A_in, W) + B
    A_out = g(Z)
    return A_out

In [4]:
def sequence(x, W1, b1, W2, b2):
    a1 = Layer(x, W1, b1, relu)
    a2 = Layer(a1, W2, b2, softmax)
    return a2

## Initialize model parameters
The model parameters are intialized randomly as follow:

In [5]:

W1 = np.random.rand(n, hidden_layer_units)
B1 = np.random.rand(1, hidden_layer_units)
W2 = np.random.rand(hidden_layer_units, num_class)
B2 = np.random.rand(1, num_class)



A2 = sequence(X_train, W1, B1, W2, B2)
print(f"A2 = {A2.shape}")

A2 = (24754, 4)


/opt/homebrew/lib/python3.11/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: overflow encountered in exp
  result = func(self.values, **kwargs)
